In [3]:
import sqlite3

'''
1. How to Create A Table?
>> CREATE TABLE Users(Name VARCHAR(128), Email VARCHAR(128))
>> CREATE TABLE IF NOT EXISTS Users(Name VARCHAR(128), Email VARCHAR(128))

2. How to Insert Data into Table?
>> INSERT INTO Users(Name, Email) VALUES('Kelly','kel@gmail.com')

3. How to Delete A Row?
>> DELETE FROM Users WHERE Email='ted@gmail.com'

4. How to Update A Entry?
>> UPDATE Users SET Name='Chris' WHERE Email='csev@gmail.com'
>> Explaination: "WHERE" Clause is Important. Because, Otherwise It'd do the changes for all the tables. Therefore, We must Narrow it down.
'''

print(f"SQLITE3 Installed Version: {sqlite3.version}")

SQLITE3 Installed Version: 2.6.0


In [4]:
"""
> CREATE TABLE Ages ( name VARCHAR(128), age INTEGER )
> DELETE FROM Ages;
> INSERT INTO Ages (name, age) VALUES ('Stewart', 28);
> INSERT INTO Ages (name, age) VALUES ('Keilan', 27);
> INSERT INTO Ages (name, age) VALUES ('Sharyn', 31);
> INSERT INTO Ages (name, age) VALUES ('Maggy', 31);
> INSERT INTO Ages (name, age) VALUES ('Mayra', 20);


> SELECT hex(name || age) AS X FROM Ages ORDER BY X 
> Returns a Table: (with Name in "Hexadecimal Values" Hoever, I've converted to ASCII for better understanding)
    |1| Keilan27  |
    |2| Maggy31   | 
    |3| Mayra20   | 
    |4| Sharyn31  | 
    |5| Stewart28 |
"""
import sqlite3, re
from collections import defaultdict


connection = sqlite3.connect("../DBs/DB1.sqlite")
cursr = connection.cursor()
cursr.execute("DROP TABLE IF EXISTS Counts")
cursr.execute("CREATE TABLE Counts (org TEXT, count INTEGER)")
mailPattern = re.compile("^From:.+@(\S+)")

orgCount = defaultdict(int)
"""
    @Question   : What is "defaultdict"
    @Answer     : 
"""
with open("../Samples/mbox.txt", mode='r') as fd:
    for line in fd:
        line = line.rstrip()
        rout = mailPattern.findall(line)
        # \S -> Means Non-Whitespace  Charater To Look for in the text file. 
        if rout:
            orgCount[rout[0]] += 1
    fd.close()

for org, count in orgCount.items():
    cursr.execute('INSERT INTO Counts (org, count) VALUES (?, ?)', (org, count))
connection.commit()

print("Counts:")
for row in cursr.execute('SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10') :
    print (str(row[0]), row[1])
cursr.close()


Counts:
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


### **Relational Database!-**
- A **Relational database** is made up of tables, rows, and columns. 
- The columns generally have a type such as **Text**, **Numeric**, or **Date data**. 


### **Relational Power**

- By removing the **replicated data** and **replacing** it with references to a single copy of each **bit of data**.
- We build a **"Web"** of information that the **Relational Database** can read through very quickly - even for **large amount of data**.

### **Join Operation**

- The **JOIN** operation **links** across **several tables** as part of **select** operations.
- You must tell the **JOIN** how to use the **keys** that make the connection between the table using an **ON** Clause. 

In [32]:
import select
import sqlite3
import os

path = "../DBs/Tracks.sqlite"

# @brief: Delete the sqlite file before initiating!
# @brief: To Avoid bad operations.

if os.path.exists(path) :
    os.remove(path)

conn = sqlite3.connect(path)
curs = conn.cursor()

'''
    @brief: We don't need the below commented line since we are working deeply ith only track database.
    @exmpl: curs.execute('CREATE Table Users(name VARCHAR(128), email VARCHAR(128))')
''' 

# Creating the Database.
curs.execute(f'CREATE TABLE Artist (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, name TEXT)')
curs.execute(f'CREATE TABLE Genres (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, name TEXT)')
curs.execute(f'CREATE TABLE Albums (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, artistId INTEGER, title TEXT)')
curs.execute(f'CREATE TABLE Tracks (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, title TEXT, albumId INTEGER, genreId INTEGER, len INTEGER, rating INTEGER, count INTEGER)')


# Filling the Database.
artists = [f"('{artist}')" for artist in ['Led/Zeplin', 'AC/DC', 'Taylor Swift', 'Drake']]
genres = [f"('{genre}')" for genre in ['Rock', 'Metal', 'Romance', 'Classical', 'K-Pop', 'J-Pop']]

genres_str = ', '.join(genres)
artists_str = ', '.join(artists)

curs.execute(f"INSERT INTO Artist (name) Values {artists_str}")
curs.execute(f"INSERT INTO Genres (name) Values {genres_str}")

curs.execute(f"INSERT INTO Albums (title, artistId) Values('Who Made Who', 2)")
curs.execute(f"INSERT INTO Albums (title, artistId) Values('IV', 1)")

curs.execute(f"INSERT INTO Tracks (title, rating, len, count, albumId, genreId) Values ('Black Dog', 5, 297, 0, 2, 1)")
curs.execute(f"INSERT INTO Tracks (title, rating, len, count, albumId, genreId) Values ('Stairway', 5, 482, 0, 2, 1)")
curs.execute(f"INSERT INTO Tracks (title, rating, len, count, albumId, genreId) Values ('About to Rock', 5, 313, 0, 1, 2)")
curs.execute(f"INSERT INTO Tracks (title, rating, len, count, albumId, genreId) Values ('Who made Who', 5, 207, 0, 1, 2)")

'''
    @Brief: Multiple Tables are Connected Using Parent Table's (Foriegn Key) -----> Child Table's (Primary Key) 
    @brief: Therefore, we'll now link the tables now!

'''
conn.commit()



for dat1 in curs.execute("SELECT Albums.title, Artist.name from Albums join Artist on Albums.artistId = Artist.id"):
    print(dat1)

print("\n------------------------------------\n")
for dat2 in curs.execute("SELECT Albums.title, Albums.artistId, Artist.id, Artist.name from Albums join Artist on Albums.artistId = Artist.id"):
    print(dat2)

print("\n------------------------------------\n")
for dat3 in curs.execute("SELECT Tracks.title, Genres.name from Tracks join Genres on Tracks.genreId = Genres.id"):
    print(dat3)

print("\n------------------------------------\n")
for dat4 in curs.execute("SELECT Tracks.title, Artist.name, Albums.title, Genres.name from Tracks join Genres join Albums join Artist on Tracks.genreId = Genres.id and Tracks.albumId = Albums.id and Albums.artistId = Artist.id"): 
    print(dat4)

('Who Made Who', 'AC/DC')
('IV', 'Led/Zeplin')

------------------------------------

('Who Made Who', 2, 2, 'AC/DC')
('IV', 1, 1, 'Led/Zeplin')

------------------------------------

('Black Dog', 'Rock')
('Stairway', 'Rock')
('About to Rock', 'Metal')
('Who made Who', 'Metal')

------------------------------------

('Black Dog', 'Led/Zeplin', 'IV', 'Rock')
('Stairway', 'Led/Zeplin', 'IV', 'Rock')
('About to Rock', 'AC/DC', 'Who Made Who', 'Metal')
('Who made Who', 'AC/DC', 'Who Made Who', 'Metal')


In [ ]:
# All Functions For Projects: 
def get_number_of_dicts(fnd):
    print(f"Dict Counter: {len(fnd)}")



In [34]:
# Project1:

from cgitb import lookup
from tkinter import NO
import xml.etree.ElementTree as et
import sqlite3, os

import xml

path_project_db = '../DBs/Project.sqlite'
if os.path.exists(path=path_project_db):
    print('Removing Old Database!')
    os.remove(path=path_project_db)

projectConn = sqlite3.connect(path_project_db)
projectCurs = projectConn.cursor()

projectCurs.executescript('''
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

fileName = '../DBs/Libs.xml'

all = et.parse(fileName)
fnd = all.findall('dict/dict/dict')
'''
    @brief: 

    - fnd = all.findall('dict/dict/dict') 
    - This Will go till 1.

    <dict>
        <key>Major Version</key><integer>1</integer>
        <key>Minor Version</key><integer>1</integer>
        <key>Date</key><date>2015-11-24T11:12:10Z</date>
        <key>Application Version</key><string>12.3.1.23</string>
        <key>Features</key><integer>5</integer>
        <key>Show Content Ratings</key><true/>
        <key>Music Folder</key><string>file:///Users/csev/Music/iTunes/iTunes%20Music/</string>
        <key>Library Persistent ID</key><string>B7006C9E9799282E</string>
        <key>Tracks</key>
        <dict>
                <key>369</key>
1--------------><dict>
2------------------><key>Track ID</key><integer>369</integer>
                        <key>Name</key><string>Another One Bites The Dust</string>

The marked element will be returned again and again!.

    - fnd = all.findall('dict/dict/dict/')
    - This will go deeper Till 2.
'''

def lookupElement(elementDict : et.Element, key: str):
    found = False
    for child in elementDict:
        if found : return child.text
        if child.tag == 'key' and child.text == key:
            found = True
    return None

def extract_dat(numbers : et.Element):
    name = artist = Genre = album = cnt = rating = length = None
    
    name = lookupElement(numbers, 'Name')
    artist = lookupElement(numbers, 'Artist')
    Genre = lookupElement(numbers, 'Genre')

    album = lookupElement(numbers, 'Album')
    cnt = lookupElement(numbers, 'Play Count')
    rating = lookupElement(numbers, 'Rating')
    length = lookupElement(numbers, 'Total Time')

    return name, artist, Genre, album, cnt, rating, length 

sum = 0
for numbers in fnd:
    name, artist, Genre, album, cnt, rating, length = extract_dat(numbers)
    if name is None or artist is None or Genre is None or album is None or cnt is None or rating is None or length is None:
        name = artist = album = cnt = rating = length = None
        sum += 1
        continue

    projectCurs.execute('INSERT OR IGNORE INTO Artist (name) Values ( ? )', (artist, ))
    projectCurs.execute('INSERT OR IGNORE INTO Genre (name) Values ( ? )', (Genre, ))
    genre_id = projectCurs.execute('SELECT id From Genre WHERE name = ?', (Genre, )).fetchone()[0]

    artist_id = projectCurs.execute('SELECT id FROM Artist WHERE name = ? ', (artist, )).fetchone()[0]
    projectCurs.execute('INSERT OR IGNORE INTO Album (title, artist_id) Values ( ?, ? )', (album, artist_id))
   
    album_id = projectCurs.execute('SELECT id FROM Album WHERE title = ? ', (album, )).fetchone()[0]
    projectCurs.execute('INSERT OR REPLACE INTO Track (title, album_id, genre_id, len, rating, count) Values (?, ?, ?, ?, ?, ?)', (name, album_id, genre_id, length, rating, cnt))

print(f'None Received!  {sum}')

projectConn.commit()
projectCurs.close()

Removing Old Database!


OperationalError: 5 values for 6 columns